In [1]:
import os
import shutil
import random

# === CONFIGURATION ===
SOURCE_DIR = r"/kaggle/input/full-dataset-20person/NEW_DataSet_20_person"  # 👈 Your dataset path
DEST_ROOT = "./kfold_dataset_14_3_3"
CLASS_MAP = {
    "Left_eye": "left",
    "Right_eye": "right",
    "Forward_eye": "forward",
    "Close_eye": "close"
}
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# === Get all person IDs
sample_class = list(CLASS_MAP.keys())[0]
sample_dir = os.path.join(SOURCE_DIR, sample_class)
all_persons = sorted([p for p in os.listdir(sample_dir) if os.path.isdir(os.path.join(sample_dir, p))])
assert len(all_persons) == 20, f"Expected 20 persons, found {len(all_persons)}"

# === Shuffle and split persons into 4 folds of 3 test persons each
random.shuffle(all_persons)
test_folds = [all_persons[i::4] for i in range(4)]  # 5 test sets, pick 3 for each

for fold_index in range(4):
    test_persons = test_folds[fold_index][:3]
    remaining = [p for p in all_persons if p not in test_persons]

    val_persons = random.sample(remaining, 3)
    train_persons = [p for p in remaining if p not in val_persons]

    print(f"\n📂 Fold {fold_index + 1}")
    print(f"   Train Persons: {train_persons}")
    print(f"   Val Persons:   {val_persons}")
    print(f"   Test Persons:  {test_persons}")

    # === Copy files into structured folders
    for split, persons in [("train", train_persons), ("val", val_persons), ("test", test_persons)]:
        for class_folder, class_name in CLASS_MAP.items():
            for person_id in persons:
                src_dir = os.path.join(SOURCE_DIR, class_folder, person_id)
                dst_dir = os.path.join(DEST_ROOT, f"fold_{fold_index + 1}", split, class_name)
                os.makedirs(dst_dir, exist_ok=True)
                for file in os.listdir(src_dir):
                    shutil.copy2(
                        os.path.join(src_dir, file),
                        os.path.join(dst_dir, file)
                    )

print("\n✅ 4 folds with 14 train / 3 val / 3 test persons each created at:", DEST_ROOT)



📂 Fold 1
   Train Persons: ['p06', 'p15', 'p05', 'p16', 'p19', 'p18', 'p11', 'p02', 'p12', 'p03', 'p17', 'p09', 'p01', 'p04']
   Val Persons:   ['p13', 'p08', 'p14']
   Test Persons:  ['p20', 'p10', 'p07']

📂 Fold 2
   Train Persons: ['p20', 'p15', 'p05', 'p10', 'p16', 'p19', 'p07', 'p18', 'p02', 'p12', 'p03', 'p08', 'p01', 'p04']
   Val Persons:   ['p09', 'p11', 'p17']
   Test Persons:  ['p06', 'p14', 'p13']

📂 Fold 3
   Train Persons: ['p06', 'p05', 'p10', 'p14', 'p07', 'p13', 'p11', 'p02', 'p12', 'p17', 'p08', 'p09', 'p01', 'p04']
   Val Persons:   ['p20', 'p19', 'p03']
   Test Persons:  ['p15', 'p16', 'p18']

📂 Fold 4
   Train Persons: ['p20', 'p06', 'p15', 'p10', 'p16', 'p07', 'p13', 'p18', 'p02', 'p03', 'p17', 'p09', 'p01', 'p04']
   Val Persons:   ['p08', 'p12', 'p14']
   Test Persons:  ['p05', 'p19', 'p11']

✅ 4 folds with 14 train / 3 val / 3 test persons each created at: ./kfold_dataset_14_3_3


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# === CONFIGURATION ===
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS_PHASE1 = 50
EPOCHS_PHASE2 = 50
FOLDS = 4
FOLD_PATH = "./kfold_dataset_14_3_3"

# === Learning rate scheduler (Cosine Decay) ===
def cosine_decay(epoch):
    max_lr = 1e-3
    min_lr = 1e-6
    return float(min_lr + 0.5 * (max_lr - min_lr) * (1 + np.cos(np.pi * epoch / EPOCHS_PHASE1)))

# === Data Augmentation ===
train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    brightness_range=[0.6, 1.4],
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_aug = ImageDataGenerator(rescale=1./255)

# === Store accuracy results
fold_accuracies = []

# === CLASS LABELS (based on directory names)
class_names = ['close', 'forward', 'left', 'right']

# === Train and evaluate on each fold
for i in range(1, FOLDS + 1):
    print(f"\n🔁 Training on Fold {i}/{FOLDS}")

    train_data = train_aug.flow_from_directory(
        os.path.join(FOLD_PATH, f"fold_{i}", "train"),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )

    val_data = val_aug.flow_from_directory(
        os.path.join(FOLD_PATH, f"fold_{i}", "val"),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    test_data = val_aug.flow_from_directory(
        os.path.join(FOLD_PATH, f"fold_{i}", "test"),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # === Build model
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False  # Phase 1 frozen

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    output = Dense(4, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )

    checkpoint1 = ModelCheckpoint(
        f'model_fold_{i}_phase1.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )

    print("🔼 Phase 1: Training classification head...")
    model.fit(
        train_data,
        validation_data=val_data,
        epochs=EPOCHS_PHASE1,
        callbacks=[
            LearningRateScheduler(cosine_decay),
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
            checkpoint1
        ],
        verbose=1
    )

    # === Phase 2: Fine-tune
    print("🔁 Phase 2: Fine-tuning top 30 base layers...")
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=2e-5),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )

    checkpoint2 = ModelCheckpoint(
        f'model_fold_{i}_fine_tuned.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )

    model.fit(
        train_data,
        validation_data=val_data,
        epochs=EPOCHS_PHASE2,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
            checkpoint2
        ],
        verbose=1
    )

    # === Evaluate on test set ===
    print("🧪 Evaluating on test set...")
    predictions = model.predict(test_data, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_data.classes

    print("📋 Classification Report:")
    report = classification_report(y_true, y_pred, target_names=class_names, digits=4)
    print(report)

    cm = confusion_matrix(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    acc = np.mean(y_true == y_pred)

    fold_accuracies.append(acc)

    # === Save confusion matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Fold {i} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_fold_{i}.png')
    plt.close()

    with open(f"classification_report_fold_{i}.txt", "w") as f:
        f.write(report)
        f.write(f"\nF1 Score: {f1:.4f}")
        f.write(f"\nAccuracy: {acc:.4f}")

    print(f"✅ Fold {i} Test Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# === Final summary
mean_acc = np.mean(fold_accuracies)
std_acc = np.std(fold_accuracies)
print(f"\n📊 Average Test Accuracy across {FOLDS} folds: {mean_acc:.4f} ± {std_acc:.4f}")


2025-05-13 20:19:35.714524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747167575.914307      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747167575.980717      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



🔁 Training on Fold 1/4
Found 2777 images belonging to 4 classes.
Found 595 images belonging to 4 classes.
Found 597 images belonging to 4 classes.


I0000 00:00:1747167590.011421      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
🔼 Phase 1: Training classification head...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

Epoch 1/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.5921 - loss: 1.0870
Epoch 1: val_accuracy improved from -inf to 0.76897, saving model to model_fold_3_phase1.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 51s 239ms/step - accuracy: 0.5927 - loss: 1.0861 - val_accuracy: 0.7690 - val_loss: 0.7533 - learning_rate: 0.0010
Epoch 2/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.8240 - loss: 0.7162
Epoch 2: val_accuracy did not improve from 0.76897
174/174 ━━━━━━━━━━━━━━━━━━━━ 32s 185ms/step - accuracy: 0.8241 - loss: 0.7161 - val_accuracy: 0.7504 - val_loss: 0.7704 - learning_rate: 9.9901e-04
Epoch 3/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.8644 - loss: 0.6395
Epoch 3: val_accuracy improved from 0.76897 to 0.81788, saving model to model_fold_3_phase1.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 32s 186ms/step - accuracy: 0.8644 - loss: 0.6395 - val_accuracy: 0.8179 - val_loss: 0.6961 - learning_rate: 9.9606e-04
Epoch 4/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/ste

E0000 00:00:1747169482.316567     116 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747169482.512508     116 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.6732 - loss: 1.1692
Epoch 1: val_accuracy improved from -inf to 0.82293, saving model to model_fold_3_fine_tuned.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 61s 245ms/step - accuracy: 0.6739 - loss: 1.1676 - val_accuracy: 0.8229 - val_loss: 0.7168
Epoch 2/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.8975 - loss: 0.6312
Epoch 2: val_accuracy did not improve from 0.82293
174/174 ━━━━━━━━━━━━━━━━━━━━ 32s 183ms/step - accuracy: 0.8976 - loss: 0.6311 - val_accuracy: 0.8094 - val_loss: 0.8512
Epoch 3/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.9267 - loss: 0.5733
Epoch 3: val_accuracy improved from 0.82293 to 0.83980, saving model to model_fold_3_fine_tuned.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 32s 186ms/step - accuracy: 0.9267 - loss: 0.5733 - val_accuracy: 0.8398 - val_loss: 0.8160
Epoch 4/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.9365 - loss: 0.5395
Epoch 4: val_accuracy did not improve from 0.83

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.5431 - loss: 1.1983
Epoch 1: val_accuracy improved from -inf to 0.77872, saving model to model_fold_4_phase1.keras
175/175 ━━━━━━━━━━━━━━━━━━━━ 47s 222ms/step - accuracy: 0.5437 - loss: 1.1970 - val_accuracy: 0.7787 - val_loss: 0.7325 - learning_rate: 0.0010
Epoch 2/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.7865 - loss: 0.7500
Epoch 2: val_accuracy improved from 0.77872 to 0.83277, saving model to model_fold_4_phase1.keras
175/175 ━━━━━━━━━━━━━━━━━━━━ 32s 184ms/step - accuracy: 0.7866 - loss: 0.7498 - val_accuracy: 0.8328 - val_loss: 0.6780 - learning_rate: 9.9901e-04
Epoch 3/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.8576 - loss: 0.6553
Epoch 3: val_accuracy improved from 0.83277 to 0.87669, saving model to model_fold_4_phase1.keras
175/175 ━━━━━━━━━━━━━━━━━━━━ 33s 187ms/step - accuracy: 0.8577 - loss: 0.6552 - val_accuracy: 0.8767 - val_loss: 0.6157 - learning_rate: 9.9606e-04
Epoch

In [3]:
import os
import tensorflow as tf

# === CONFIGURATION ===
KERAS_MODEL_DIR = "./"  # Folder containing your .keras files
TFLITE_OUTPUT_DIR = "./tflite_models"
os.makedirs(TFLITE_OUTPUT_DIR, exist_ok=True)

# === Convert all .keras models
keras_models = [f for f in os.listdir(KERAS_MODEL_DIR) if f.endswith(".keras")]

for keras_file in keras_models:
    keras_path = os.path.join(KERAS_MODEL_DIR, keras_file)
    tflite_filename = keras_file.replace(".keras", ".tflite")
    tflite_path = os.path.join(TFLITE_OUTPUT_DIR, tflite_filename)

    print(f"🔄 Converting: {keras_file} → {tflite_filename}")

    # Load model
    model = tf.keras.models.load_model(keras_path)

    # Convert to TFLite (FP32)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    # Optimize for size/speed (optional but recommended for Raspberry Pi)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # Convert
    tflite_model = converter.convert()

    # Save
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)

    print(f"✅ Saved: {tflite_path}")

print("\n📁 All models converted to:", TFLITE_OUTPUT_DIR)


🔄 Converting: model_fold_4_fine_tuned.keras → model_fold_4_fine_tuned.tflite
Saved artifact at '/tmp/tmp2zbfpze9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137106027719440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106027711952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106027723280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106027715600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106027712528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106027716752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106033887952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106033886224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106027723856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137106

W0000 00:00:1747170676.295059      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747170676.295110      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1747170676.436180      35 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


✅ Saved: ./tflite_models/model_fold_4_fine_tuned.tflite
🔄 Converting: model_fold_2_fine_tuned.keras → model_fold_2_fine_tuned.tflite
Saved artifact at '/tmp/tmp7uqspd5d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137098182471568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182475600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182475024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182475216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182471376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182471760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182473104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182472720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098182472912: Ten

W0000 00:00:1747170701.441516      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747170701.441551      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


✅ Saved: ./tflite_models/model_fold_2_fine_tuned.tflite
🔄 Converting: model_fold_3_phase1.keras → model_fold_3_phase1.tflite
Saved artifact at '/tmp/tmp2lm4e0hm'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137099997887376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997886032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997885648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997885840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997887184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997887568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997884112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997883728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137099997883920: TensorSpec(

...

W0000 00:00:1747170801.475529      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747170801.475565      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


✅ Saved: ./tflite_models/model_fold_2_phase1.tflite
🔄 Converting: model_fold_4_phase1.keras → model_fold_4_phase1.tflite
Saved artifact at '/tmp/tmp_6v4_bkq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137098192698448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192697104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192696720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192696912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192698256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192698640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192694800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192694416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137098192694608: TensorSpec(shap

W0000 00:00:1747170826.703567      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747170826.703606      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


✅ Saved: ./tflite_models/model_fold_4_phase1.tflite
🔄 Converting: model_fold_1_fine_tuned.keras → model_fold_1_fine_tuned.tflite
Saved artifact at '/tmp/tmp28gpyltp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137100124830800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124829456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124829072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124829264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124830608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124830992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124827152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124826768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137100124826960: TensorSpe

W0000 00:00:1747170851.428508      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1747170851.428551      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
